In [1]:
import pennylane as qml

In [2]:
U1 = qml.prod(qml.PauliX(0), qml.PauliZ(1))
U2 = qml.prod(qml.Identity(0), qml.PauliZ(1))

U = qml.s_prod(1/2, qml.sum(U1, U2))  # U = 0.5 * (U1 + U2)

print(qml.matrix(U))

[[ 0.5  0.   0.5  0. ]
 [ 0.  -0.5  0.  -0.5]
 [ 0.5  0.   0.5  0. ]
 [ 0.  -0.5  0.  -0.5]]


In [3]:
import numpy as np
# Set up core operators: 

def Prep():
    qml.Hadamard(wires='Prep_Register')
    
    
def Sel():
    qml.ctrl(U1, control=['Prep_Register'], control_values=0)
    qml.ctrl(U2, control=['Prep_Register'], control_values=1)


# Test: 
dev = qml.device("default.qubit", wires=['Prep_Register', 0, 1])

@qml.qnode(dev)
def circ():
    Prep()
    Sel()
    qml.adjoint(Prep)()
    
    return qml.state()

top_left_block = qml.matrix(circ)()[:4, :4]

print(qml.draw(circ)())
print("\n", top_left_block)

print("\n", np.allclose(top_left_block, qml.matrix(U)))

Prep_Register: ──H─╭○───╭●────H†─┤  State
            0: ────├X@Z─├I@Z─────┤  State
            1: ────╰X@Z─╰I@Z─────┤  State

 [[ 0.5+0.j  0. +0.j  0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j  0. +0.j -0.5+0.j]
 [ 0.5+0.j  0. +0.j  0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j  0. +0.j -0.5+0.j]]

 True
